<a href="https://colab.research.google.com/github/IvanOM-97/DPro-Exercises/blob/master/U32T1C61CreditInformationLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
APRENDIZAJE DE INFORMACION CREDITICIA
    Aprender el uso de los procesos básicos de aprendizaje automático y el manejo de librerías aprovechando conjuntos de datos de Kaggle.
    A través de la competencia Home Credit Default Risk en Kaggle, se realizará entrenamiento y estimación.
      https://www.kaggle.com/c/home-credit-default-risk
'''

In [ ]:
'''
PROBLEMA 1 - COMFIRMACION DEL CONTENIDO DE LA COMPETENCIA
Confirmacion del contenido de la competencia:
  Qué aprender y qué predecir
    las informacion de transacciondes de los clientes y sus posibilidad de pago
  Qué archivo crear y enviar a Kaggle
    por cada SK_ID_CURR en el set de test, debe predecir la probabilidad para la variable objetivo
    el archivo dede contener un encabezado y tener el sgte formato
      SK_ID_CURR,TARGET
      100001,0.1
      100002,0.33
      100003,0.75
  ¿Qué tipo de valor de índice se evaluarán los elementos enviados?
    loe elementos enviados seran evaluados en el area bajo la curva ROC entre la probabilidad predicha y el objetivo observado
'''


In [ ]:
# Ejecutar primero para instalar esta libreria
!pip install category_encoders

In [ ]:
'''
PROBLEMA 2 - APREDIZAJE Y VERIFICACION
  Después de analizar los datos de manera simple y aplicar el preprocesamiento, cree y ejecute el flujo de entrenamiento y validación. Procure utilizar las métricas de evaluación que se emplean en esta competición para la validación.
  No se especifica ningún método particular para el entrenamiento.
  Por ejemplo, en problemas de clasificación se utiliza frecuentemente la regresión logística, y en la práctica profesional también se adopta como una fase inicial simple.
'''
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from category_encoders import CountEncoder

# Import the csv of the dataset
df = pd.read_csv('application_train.csv')

# cleaning the dataset by removing the empity data
cleaned_df = df.dropna()
categorical_feats = df.select_dtypes('object').columns.tolist()

# Separating them into variables
X = df.drop(columns = ['TARGET'])
y = df['TARGET']

# Encoding values
X = CountEncoder(cols = categorical_feats).fit_transform(X)

# Sliptting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Standardizing the data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_trans = scaler.transform(X_train)
X_test_trans = scaler.transform(X_test)

# Fiting the data
from lightgbm import LGBMClassifier

reg = LGBMClassifier(random_state = 5, verbosity=-1).fit(X_train_trans, y_train)

# Predicting the data
reg_pred = reg.predict(X_test_trans)

print("Acc: ", accuracy_score(y_true = y_test, y_pred = reg_pred))
print("AUC: ", roc_auc_score(y_test, reg_pred))


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Acc:  0.9196136773815912
AUC:  0.5087100094765098


In [ ]:
'''
PROBLEMA 3 - ESTIMACION PARA DATOS DE PRUEBA
  Realice predicciones en los datos de prueba (application_test.csv) y envíelas a Kaggle.
  Si el formato de envío es correcto, la precisión no es tan importante.
  El objetivo en esta etapa inicial es experimentar el flujo de envío y obtener retroalimentación.
'''
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from category_encoders import CountEncoder
from lightgbm import LGBMClassifier

# Import the csv of the dataset
df = pd.read_csv('application_train.csv')

# cleaning the dataset by removing the empity data
cleaned_df = df.dropna()
categorical_feats = df.select_dtypes('object').columns.tolist()

# Separating them into variables
X = df.drop(columns = ['TARGET'])
y = df['TARGET']

# Encoding values
X = CountEncoder(cols = categorical_feats).fit_transform(X)

# Sliptting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Standardizing the data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_trans = scaler.transform(X_train)
X_test_trans = scaler.transform(X_test)

# Fiting the data
reg = LGBMClassifier(random_state = 5, verbosity=-1).fit(X_train_trans, y_train)

# Predicting the data
reg_pred = reg.predict(X_test_trans)

# Loading the csv of the test dataset
test_df = pd.read_csv('application_test.csv')

# Cleaning th dataset by removing the empy data(null)
test_cleaned_df = test_df.dropna(axis = 0)

# Encoding values
test_X = X = CountEncoder(cols = categorical_feats).fit_transform(test_df)

# Standardizing the data
test_scaler = StandardScaler()
test_X_test_trans = scaler.fit_transform(test_X)

# Predicting the data
test_reg_pred = reg.predict(test_X_test_trans)

kgl_submission = pd.concat([test_df['SK_ID_CURR'], pd.Series(test_reg_pred, name = 'TARGET')], axis = 1)
kgl_submission.to_csv('submission.csv', index = False)
kgl_submission.head()

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


,SK_ID_CURR,TARGET
0,100001,0
1,100005,0
2,100013,0
3,100028,0
4,100038,0


In [ ]:
'''
PROBLEMA 4 - INGENIERIA DE CARACTERISTICAS
  Proceda con la ingeniería de características (Feature Engineering) desde las siguientes perspectivas e intente mejorar la precisión del modelo:
      - Seleccionar qué características utilizar
      - Decidir cómo preprocesar las características
  Registre cómo los procesos realizados afectaron las métricas de evaluación, e implemente al menos 5 patrones de entrenamiento y validación. Para el modelo que finalmente obtuvo el mejor rendimiento, realice también predicciones en los datos de prueba y envíelas a Kaggle.
'''
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from category_encoders import CountEncoder
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMClassifier

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# LOAD DATA
df = pd.read_csv('application_train_copia.csv')

# Separate features and target
X = df.drop(columns=['TARGET'])
y = df['TARGET']

# PREPROCESSING FUNCTIONS
def preprocess_numerical_data(X_input):
    """Impute and scale numerical features."""
    X_numerical = X_input.select_dtypes(include=np.number)
    imputer = SimpleImputer(strategy='median')
    X_num_imputed = imputer.fit_transform(X_numerical)
    scaler = StandardScaler()
    X_num_scaled = scaler.fit_transform(X_num_imputed)
    return X_num_scaled

def preprocess_categorical_data(X_input):
    """Impute and one-hot encode categorical features."""
    X_categorical = X_input.select_dtypes(exclude=np.number)
    imputer = SimpleImputer(strategy='most_frequent')
    X_cat_imputed = imputer.fit_transform(X_categorical)
    encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    X_cat_encoded = encoder.fit_transform(X_cat_imputed)
    return X_cat_encoded

# Combine numerical and categorical preprocessed data
X_preprocessed = np.hstack((preprocess_numerical_data(X), preprocess_categorical_data(X)))

# MODEL FUNCTIONS
def logistic_regression(X_train, y_train, X_test, y_test):
    model = LogisticRegression(max_iter=500)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    return accuracy_score(y_test, pred), roc_auc_score(y_test, pred)

def linear_regression_model(X_train, y_train, X_test, y_test):
    model = LinearRegression()
    model.fit(X_train, y_train)
    pred = np.round(model.predict(X_test))  # Round predictions to 0 or 1
    return accuracy_score(y_test, pred), roc_auc_score(y_test, pred)

def svm_linear(X_train, y_train, X_test, y_test):
    model = SVC(kernel='linear')
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    return accuracy_score(y_test, pred), roc_auc_score(y_test, pred)

def svm_rbf(X_train, y_train, X_test, y_test):
    model = SVC(kernel='rbf')
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    return accuracy_score(y_test, pred), roc_auc_score(y_test, pred)

def decision_tree(X_train, y_train, X_test, y_test):
    model = DecisionTreeClassifier(max_depth=5)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    return accuracy_score(y_test, pred), roc_auc_score(y_test, pred)

def random_forest(X_train, y_train, X_test, y_test):
    model = RandomForestClassifier(max_depth=10, n_estimators=50)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    return accuracy_score(y_test, pred), roc_auc_score(y_test, pred)

# TRAINING AND VALIDATION
models = {
    "Logistic Regression": logistic_regression,
    "Linear Regression": linear_regression_model,
    "SVM (Linear)": svm_linear,
    "SVM (RBF)": svm_rbf,
    "Decision Tree": decision_tree,
    "Random Forest": random_forest
}

results = []

# Try 5 different random splits for training/validation
for seed in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.25, random_state=seed)
    print(f"\n=== Training round {seed+1} ===")
    for name, func in models.items():
        acc, auc = func(X_train, y_train, X_test, y_test)
        print(f"{name} -> Accuracy: {acc:.4f}, AUC: {auc:.4f}")
        results.append([seed+1, name, acc, auc])

# Convert results to DataFrame
df_results = pd.DataFrame(results, columns=["Round", "Model", "Accuracy", "AUC"])
print("\nSummary of results:")
print(df_results)

# FINAL PREDICTION (Best Model)
# Select the model with the best average AUC
best_model_name = df_results.groupby("Model")["AUC"].mean().idxmax()
print(f"\nBest model based on average AUC: {best_model_name}")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Acc:  0.9195738702879888
AUC:  0.5071307277099971
